In [1]:
import numpy as np
import math
import copy
import time
from scipy.stats import ks_2samp

In [2]:
#Open + read file
a = open('multidim-data.txt', 'r')
sample = a.readlines()
sample = [x.strip('\n') for x in sample] 
sample = [x.split(',') for x in sample] 

#Delete all the attributes you don't care about
for i, asdf in enumerate(sample):
    del(asdf[7])
    del(asdf[0:4])
    
#Turn strings into floats
for i in range(len(sample)):
    for j in range(len(sample[i])):
        sample[i][j] = float(sample[i][j])
        


In [3]:
#This assumes that your sample is currently an array of arrays, each array corresponding to an element and its elements being the
#values of the variables as floats.

#you have to define this global variable before you can run the rest of it.
numDivs = 5
    
    
def sample_to_pdf(sample, numObjects): #sample is the sample that we wish to take a subsample from and numobjects is the number
#of groupings we wish to create for the main function.
    m = len(sample[0])
    n = len(sample)
    
    #Compute extrema for binning
    maxValues = np.zeros((m,))
    for i in range(m):
        maxValues[i] = -float('inf')
    for i in sample:
        for j in range(len(i)):
            if i[j] > maxValues[j]:
                maxValues[j] = i[j]
    for i in range(m):
        maxValues[i] += 0.0001

    minValues = np.zeros((m,))
    for i in range(m):
        minValues[i] = float('inf')
    for i in sample:
        for j in range(len(i)):
            if i[j] < minValues[j]:
                minValues[j] = i[j]
    
    

    binSize = (maxValues - minValues) / numDivs
    #Create the bins and bin the sample so we get our pdfs as well as the overall pdf.
    tempList = [numObjects]
    tempList.extend([numDivs for i in range(m)])
    distributions = list(np.zeros(tempList))
    overallPDF = np.zeros([numDivs for i in range(m)])
    objectContents = [set() for i in range(numObjects)]
    objectDistribution = np.array([1.04**(j) for j in range(numObjects)])
    objectDistribution = objectDistribution/sum(objectDistribution)
    for k, element in enumerate(sample):
        i = np.random.choice(numObjects, p = objectDistribution)
        objectContents[i].add(k)
        pos = [0 for l in range(m)]
        for j in range(m):
            pos[j] = math.floor((element[j]-minValues[j]) / binSize[j])
        positions = tuple(pos)
        distributions[i][positions]+= 1
        overallPDF[positions]+=1
    weights = list(np.zeros((numObjects,)))
    i = 0
    numSkipped = 0
    while i + numSkipped < numObjects:
        weights[i] = np.sum(distributions[i])
        if weights[i] == 0:
            del(weights[i])
            del(distributions[i])
            del(objectContents[i])
            numSkipped += 1
        else:
            i += 1
    overallPDF = overallPDF / np.sum(overallPDF)
    for i, split in enumerate(distributions):
        tempSum = np.sum(split)
        distributions[i] = split / tempSum
    return weights, distributions, overallPDF, objectContents

In [4]:
#Recursive helper function to give us all indices so that we can get the values correctly for the multidimensional case.
#You should use this to set a global variable before you need to use it so you only need to call this once. If you call this
#function many times with many dimensions, it will slow down your code a lot.

def nd_range(start, stop, dims):
    if not dims:
        yield ()
        return
    for outer in nd_range(start, stop, dims - 1):
        for inner in range(start, stop):
            yield outer + (inner,)
            
def find_depth(List):
    if type(List) == list or type(List) == np.ndarray:
        return find_depth(List[0]) + 1
    else:
        return 0
    
def hellinger_dist(sampleDist, PDF):
    coefficient = 0
    tempDist = copy.deepcopy(sampleDist)
    totalSum = np.sum(sampleDist)
    if totalSum > 0:
        for i in indices:
            tempDist[i] = tempDist[i] / totalSum
        for i in indices:
            coefficient += math.sqrt(tempDist[i]*PDF[i])
        return math.sqrt(1-coefficient)
    else:
        return 0


In [31]:
w, p, overallPDF, objectContents =sample_to_pdf(sample, 200)
maxWeight = 12000
m = find_depth(p) - 1
indices = list(nd_range(0, numDivs, m))

In [6]:
z = np.concatenate(p)
z.shape

(1000, 5, 5)

In [32]:
"""
A simple version of the algorithm. The goal is to split a dataset into a number of representative samples such that the sample 
size is upper bounded by a constant. Furthermore, we want each sample to have a similar number of elements and objects in it.
"""

def bin_packing_ffd_mod(weights, pdfs, max_size, distance_func):
    avgWeight = sum(weights)/len(weights)
    n = len(weights)
    sample0 = overallPDF
    inds_sorted = np.argsort(weights)[::-1]
    #print(inds_sorted)
    inds2 = list(inds_sorted)
    weights2 = list(weights[inds2[i]] for i in range(n))
    pdfs2 = [pdfs[inds2[i]] for i in range(n)]
    
    bins = [[]]
    r_pdfs = [[]]
    ind_cur_bin = 0
    if weights2[0] > max_size:
        return False, []
    improves_pdf = True

    lower_bound_bins_number = int(round(sum(weights) / max_size + 0.5))
    bins = [[x] for x in weights2[:lower_bound_bins_number]]
    r_pdfs = [x*weights2[i] for i, x in enumerate(pdfs2[:lower_bound_bins_number])]
    indices = [[i] for i in inds2[:lower_bound_bins_number]]

    weights2 = weights2[lower_bound_bins_number:]
    pdfs2 = pdfs2[lower_bound_bins_number:]
    inds2 = inds2[lower_bound_bins_number:]
    while weights2:
        dispatched = False
        cnt = 1
        ind_cur_ssample = 0
        while not dispatched:
            cur_bin = bins[ind_cur_bin]
            cur_pdf_bin = r_pdfs[ind_cur_bin]
            cur_ind_bin = indices[ind_cur_bin]
            
            
            ks_cur = distance_func(cur_pdf_bin, sample0)
            ks_cur2 = distance_func(cur_pdf_bin + weights2[ind_cur_ssample]*pdfs2[ind_cur_ssample], sample0)
            # print(sample0.shape, ks_cur, ks_cur2)
            #print(cur_bin)
            #sum(cur_bin)*np.random.random()/(len(cur_bin)*max_size)
            accepted = (len(cur_bin)*sum(cur_bin)*np.random.random()/((n/len(bins))*max_size) < ks_cur2 - ks_cur + 1 - sum(cur_bin)/max_size)
            if max_size - sum(cur_bin) >= weights2[ind_cur_ssample] and accepted:
                cur_pdf_bin += pdfs2.pop(ind_cur_ssample)*weights2[ind_cur_ssample]
                cur_bin.append(weights2.pop(ind_cur_ssample))
                cur_ind_bin.append(inds2.pop(ind_cur_ssample))
                dispatched = True
            elif cnt < 20*len(bins):
                cnt += 1
                ind_cur_bin = (ind_cur_bin + 1) % len(bins)
            else:
                if ind_cur_ssample < len(pdfs2) - 1:
                    ind_cur_ssample += 1
                else:
                    print("making new bin")
                    cur_bin = []
                    cur_pdf_bin = []
                    cur_ind_bin = []
                    ind_cur_ssample = 0
                    cur_pdf_bin += pdfs2.pop(ind_cur_ssample)*weights2[ind_cur_ssample]
                    cur_bin.append(weights2.pop(ind_cur_ssample))
                    cur_ind_bin.append(inds2.pop(ind_cur_ssample))
                    bins.insert(ind_cur_bin, cur_bin)
                    r_pdfs.insert(ind_cur_bin, cur_pdf_bin)
                    indices.insert(ind_cur_bin, cur_ind_bin)
                    dispatched = True
    return True, bins, r_pdfs, indices

In [33]:
a, b, c, d = bin_packing_ffd_mod(w, p, maxWeight, distance_func=hellinger_dist)

In [34]:
numBins = len(b)
print(numBins)
totalSum = 0
for i in b:
    print(sum(i))
    print(len(i))
    totalSum += sum(i)

12
11722.0
8
11425.0
12
11751.0
10
10969.0
13
11702.0
15
11171.0
16
11709.0
11
11338.0
15
11323.0
20
11222.0
17
10090.0
28
10102.0
32


In [35]:
avgDeviation = 0
worstDeviation = 0
for i in range(numBins):
    x = hellinger_dist(c[i], overallPDF)
    avgDeviation += x/(numBins)
    if x > worstDeviation:
        worstDeviation = x
print(worstDeviation)
print(avgDeviation)

0.038676722464610125
0.034666344497023646


In [34]:
np.argsort(w)

array([  3,  16,   0,  22,  15,  14,   2,   7,   9,  19,  17,  13,  25,
         8,   6,   5,   4,  11,  10,  21,   1,  12,  30,  36,  18,  23,
        26,  27,  20,  39,  42,  29,  34,  45,  41,  24,  40,  54,  33,
        32,  44,  37,  59,  43,  46,  58,  31,  35,  28,  47,  56,  52,
        38,  49,  51,  60,  55,  53,  50,  57,  63,  61,  48,  64,  75,
        71,  62,  65,  67,  73,  69,  70,  66,  68,  74,  83,  76,  77,
        72,  80,  79,  85,  78,  86,  87,  82,  81,  94,  89,  93,  84,
        90,  88,  91,  96,  97,  92,  95,  98, 101, 103, 100, 102, 106,
       104,  99, 105, 107, 109, 112, 110, 108, 111, 115, 114, 113, 116,
       117, 118, 119, 120, 122, 123, 124, 121, 127, 126, 128, 125, 129,
       130, 132, 131, 133, 134, 136, 137, 135, 138, 139, 141, 140, 142,
       143, 144, 145, 147, 148, 146, 149, 150, 152, 151, 153, 156, 155,
       154, 157, 158, 159, 160, 161, 162, 164, 163, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 180, 17

In [13]:
a = [1,2,3,4]

In [14]:
a.insert(2, 5)

In [15]:
a

[1, 2, 5, 3, 4]

In [16]:
len(c[0])

16

In [43]:
np.sum(overallPDF)

1.0

In [36]:
"""
This time, we reorder the bins every time we add an object so that the bins are considered in order of increasing weight, making
an object more likely to be placed in the bins that are lighter. This makes it so that the weights are more even, but more
importantly, that the number of objects in each bin is also about the same. This is because we consider the objects in decreasing
order of weight, which means that the objects added close to eachother in the process are also similar in weight, meaning that
ordering by weight also roughly orders the samples by the number of objects in them.
"""

def bin_packing_ffd_mod2(weights, pdfs, max_size, distance_func):
    avgWeight = sum(weights)/len(weights)
    n = len(weights)
    sample0 = overallPDF
    inds_sorted = np.argsort(weights)[::-1]
    #print(inds_sorted)
    inds2 = list(inds_sorted)
    weights2 = list(weights[inds2[i]] for i in range(n))
    pdfs2 = [pdfs[inds2[i]] for i in range(n)]
    
    bins = [[]]
    r_pdfs = [[]]
    ind_cur_bin = 0
    if weights2[0] > max_size:
        return False, []
    improves_pdf = True

    lower_bound_bins_number = int(round(sum(weights) / max_size + 0.5))
    bins = [[x] for x in weights2[:lower_bound_bins_number]]
    r_pdfs = [x*weights2[i] for i, x in enumerate(pdfs2[:lower_bound_bins_number])]
    indices = [[i] for i in inds2[:lower_bound_bins_number]]

    weights2 = weights2[lower_bound_bins_number:]
    pdfs2 = pdfs2[lower_bound_bins_number:]
    inds2 = inds2[lower_bound_bins_number:]
    while weights2:
        dispatched = False
        cnt = 1
        ind_cur_ssample = 0
        binWeights = [sum(bins[i]) for i in range(len(bins))]
        binOrder = np.argsort(binWeights)
        cur_bin_num = 0
        ind_cur_bin = binOrder[cur_bin_num]
        while not dispatched:
            cur_bin = bins[ind_cur_bin]
            cur_pdf_bin = r_pdfs[ind_cur_bin]
            cur_ind_bin = indices[ind_cur_bin]
            
            
            ks_cur = distance_func(cur_pdf_bin, sample0)
            ks_cur2 = distance_func(cur_pdf_bin + weights2[ind_cur_ssample]*pdfs2[ind_cur_ssample], sample0)
            # print(sample0.shape, ks_cur, ks_cur2)
            #print(cur_bin)
            #sum(cur_bin)*np.random.random()/(len(cur_bin)*max_size)
            accepted = (len(cur_bin)*sum(cur_bin)*np.random.random()/((n/len(bins))*max_size) < ks_cur2 - ks_cur + 1 - sum(cur_bin)/max_size)
            if max_size - sum(cur_bin) >= weights2[ind_cur_ssample] and accepted:
                cur_pdf_bin += pdfs2.pop(ind_cur_ssample)*weights2[ind_cur_ssample]
                cur_bin.append(weights2.pop(ind_cur_ssample))
                cur_ind_bin.append(inds2.pop(ind_cur_ssample))
                dispatched = True
            elif cnt < 10*len(bins):
                cnt += 1
                cur_bin_num = (cur_bin_num + 1) % len(bins)
                ind_cur_bin = binOrder[cur_bin_num]
            else:
                print("making new bin")
                cur_bin = []
                cur_pdf_bin = []
                cur_ind_bin = []
                ind_cur_ssample = 0
                cur_pdf_bin += pdfs2.pop(ind_cur_ssample)*weights2[ind_cur_ssample]
                cur_bin.append(weights2.pop(ind_cur_ssample))
                cur_ind_bin.append(inds2.pop(ind_cur_ssample))
                bins.insert(ind_cur_bin, cur_bin)
                r_pdfs.insert(ind_cur_bin, cur_pdf_bin)
                indices.insert(ind_cur_bin, cur_ind_bin)
                dispatched = True
    return True, bins, r_pdfs, indices

In [37]:
a, b, c, d = bin_packing_ffd_mod2(w, p, maxWeight, distance_func=hellinger_dist)

In [38]:
numBins = len(b)
print(numBins)
totalSum = 0
for i in b:
    print(sum(i))
    print(len(i))
    totalSum += sum(i)

12
11178.0
18
11193.0
18
11201.0
18
11204.0
15
11243.0
17
11264.0
17
11189.0
17
11190.0
14
11199.0
15
11272.0
15
11201.0
16
11190.0
17


In [39]:
avgDeviation = 0
worstDeviation = 0
for i in range(numBins):
    x = hellinger_dist(c[i], overallPDF)
    avgDeviation += x/(numBins)
    if x > worstDeviation:
        worstDeviation = x
print(worstDeviation)
print(avgDeviation)

0.03840824440305688
0.034208071516931754


As can be seen above, when you reorder the bins every time in order of increasing weights, we can about the same distances from the overall distribution as when we did the FFD bin packing method, but the weights of the bins and the number of objects in the bins is far more similar than with the FFD method.